In [10]:
import pandas as pd
import fastai
from fastai.tabular.all import *

c:\Users\Wiggles\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('matchdata.csv')

In [5]:
df.head()

,Unnamed: 0,BlueTop,BlueJungle,BlueMid,BlueADC,BlueSupport,RedTop,RedJungle,RedMid,RedADC,RedSupport,BlueWin
0,0,DrMundo,Amumu,Akali,MissFortune,Lux,Olaf,Sejuani,Fizz,Caitlyn,Blitzcrank,False
1,1,DrMundo,XinZhao,Kassadin,Caitlyn,Seraphine,Yorick,Trundle,Sylas,Ezreal,Karma,False
2,2,DrMundo,Vi,Kassadin,Ashe,Lux,Garen,Warwick,Akshan,Ezreal,TwistedFate,True
3,3,DrMundo,XinZhao,Zed,Kaisa,Heimerdinger,Sett,Udyr,Akali,Ashe,Brand,False
4,4,DrMundo,Amumu,Irelia,Ashe,Blitzcrank,KSante,Skarner,Zed,Nilah,Rakan,False


In [8]:
#df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,BlueTop,BlueJungle,BlueMid,BlueADC,BlueSupport,RedTop,RedJungle,RedMid,RedADC,RedSupport,BlueWin
0,DrMundo,Amumu,Akali,MissFortune,Lux,Olaf,Sejuani,Fizz,Caitlyn,Blitzcrank,False
1,DrMundo,XinZhao,Kassadin,Caitlyn,Seraphine,Yorick,Trundle,Sylas,Ezreal,Karma,False
2,DrMundo,Vi,Kassadin,Ashe,Lux,Garen,Warwick,Akshan,Ezreal,TwistedFate,True
3,DrMundo,XinZhao,Zed,Kaisa,Heimerdinger,Sett,Udyr,Akali,Ashe,Brand,False
4,DrMundo,Amumu,Irelia,Ashe,Blitzcrank,KSante,Skarner,Zed,Nilah,Rakan,False


In [27]:
df.groupby('BlueWin')['BlueWin'].count()

BlueWin
False    12556
True     11941
Name: BlueWin, dtype: int64

In [47]:
totalBlueWR = 11941 / (12556 + 11941)
print(totalBlueWR)

0.487447442543985


In [48]:
dls = TabularDataLoaders.from_csv('matchdata.csv', path='.', y_names="BlueWin", cat_names=['BlueTop', 'BlueJungle', 'BlueMid', 'BlueADC', 'BlueSupport', 'RedTop', 'RedJungle', 'RedMid', 'RedADC', 'RedSupport'], procs =[Categorify])

In [49]:
splits = RandomSplitter(valid_pct = .3)(range_of(df))

In [50]:
to = TabularPandas(df, procs=[Categorify], cat_names=['BlueTop', 'BlueJungle', 'BlueMid', 'BlueADC', 'BlueSupport', 'RedTop', 'RedJungle', 'RedMid', 'RedADC', 'RedSupport'], y_names="BlueWin", splits=splits)

In [57]:
dls = to.dataloaders(bs=64)

In [58]:
learn = tabular_learner(dls, metrics=accuracy)

In [59]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.696104,0.688388,0.560620,00:02
1,0.664885,0.683649,0.568513,00:02
2,0.631634,0.698828,0.577494,00:02
3,0.541834,0.745795,0.578582,00:02
4,0.428154,0.834201,0.587563,00:02
5,0.265665,1.034371,0.577221,00:02
6,0.140084,1.218599,0.589060,00:02
7,0.058680,1.378125,0.594367,00:02
8,0.031952,1.478962,0.590012,00:02
9,0.019031,1.496248,0.593958,00:02


So far, I've tried batch sizes ranging from 16 to 128, and epochs from 10 to 100, and larger validation percentages. Regardless, it seems that there's a huge overfitting problem that I feel is due to the fact that league is such a varied game and draft is such a small part of what makes a team win and lose. Even then, there is P(161,10) > 10^21 team compositions that can be drafted against each other and 25,000 samples is only a small snippet of that. To fix the overfitting, I have increased batch size, lowered epochs and lowered validation percent.

In [61]:
learn.show_results()

,BlueTop,BlueJungle,BlueMid,BlueADC,BlueSupport,RedTop,RedJungle,RedMid,RedADC,RedSupport,BlueWin,BlueWin_pred
0,105,77,157,87,120,36,148,73,30,155,1,1
1,74,77,73,10,68,129,144,122,53,128,1,0
2,40,29,53,26,120,48,76,142,30,128,1,1
3,35,41,3,149,98,112,119,150,71,82,1,0
4,129,117,107,65,155,90,148,91,54,68,0,0
5,93,76,55,10,135,105,39,34,50,141,1,0
6,44,132,75,18,160,35,88,12,138,81,0,0
7,133,144,2,118,121,64,77,146,51,110,1,0
8,107,13,135,19,159,2,153,34,146,47,0,1


Since it is impossible to predict how a League of Legends game will go based solely off draft alone, I want this tool to be used to know what the chances of winning/losing are.